In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
image_fn = str(Path("../../../data/Town04_Clear_Noon_09_09_2020_14_57_22_frame_625_validation_set.png"))
image = cv2.imread(image_fn)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
image.shape

In [ ]:
run_student_code = False

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(str(Path('../../')))


In [ ]:
if run_student_code:
    from exercises.camera_calibration.calibrated_lane_detector import CalibratedLaneDetector
else:
    from solutions.camera_calibration.calibrated_lane_detector import CalibratedLaneDetector

In [ ]:
# TODO: Change the next line(s), to create an instance of *your* LaneDetector
model_path = Path("../../solutions/lane_detection/best_model_multi_dice_loss.pth")
ld = CalibratedLaneDetector(model_path=model_path)

In [ ]:
ld.calibrate(image,mpl_axis = plt.gca())